<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Solving Tasks Requiring Web Info
(Web 搜尋：解決需要 Web 資訊的任務)
AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to perform tasks which require acquiring info from the web:
* discuss a paper based on its URL.
* discuss about the stock market.

Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "TERMINATE", the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`, and prompt the user for feedback when the task is finished. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent`.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install pyautogen and docker:
```bash
pip install pyautogen docker
```

In [ ]:
# %pip install "pyautogen>=0.2.3" docker

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [7]:

import os
import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent
"""
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}
"""
"""
20240219，此處列表更新為5類
gpt35、gpt4、gpt4v、gemini、gemini_vision
"""
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-16k-0613"],
    },
)
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0613", "gpt-4-0314", "gpt-4-1106-preview"],
    },
)
config_list_gpt4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-vision-preview", "dalle"],
    },
)
config_list_gemini = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro"],
    },
)
config_list_gemini_vision = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro-vision"],
    },
)

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4-32k',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-08-01-preview',
    },
    {
        'model': 'gpt-4-32k-0314',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-08-01-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for human feedback when it receives a "TERMINATE" signal from the assistant agent.

In [8]:
# 此區塊命令用途，詢問 GPT
"""
這段代碼是使用列表推導式對一系列配置字典中的鍵 "api_version" 進行移除操作。
讓我們來分解這段代碼：
[config.pop("api_version", None) for config in config_list_gpt4]：這行代碼對名為 config_list_gpt4 的列表中的每個配置字典 config 都執行了一個操作，即移除其中的 "api_version" 鍵。如果 "api_version" 鍵不存在於字典中，則不執行任何操作，這就是 pop 方法的作用。列表推導式的結果不會被使用，所以在這個例子中，主要是為了執行 pop 操作。
其餘的行 (config_list_gpt4v, config_list_gpt35, config_list_gemini, config_list_gemini_vision) 也是類似的，只是對應不同的配置列表。
總體來說，這段代碼的作用是從一系列配置字典中移除名為 "api_version" 的鍵，這可能是為了在後續的程式中不再使用該配置參數。
"""
[config.pop("api_version", None) for config in config_list_gpt4]
[config.pop("api_version", None) for config in config_list_gpt4v]
[config.pop("api_version", None) for config in config_list_gpt35]
[config.pop("api_version", None) for config in config_list_gemini]
[config.pop("api_version", None) for config in config_list_gemini_vision]

[None]

In [9]:
# create an AssistantAgent instance named "assistant"
"""assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
"""
assistant = AssistantAgent(
    "assistant", llm_config={"config_list": config_list_gpt35, "seed": 42}, max_consecutive_auto_reply=3
)
# 
# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    # is_termination_msg=lambda x: content_str(x.get("content")).find("TERMINATE") >= 0,
    # code_execution_config={"work_dir": "web"},
    code_execution_config={"work_dir": "coding", "use_docker": False},
    llm_config={"config_list": config_list_gpt35, "seed": 42},
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)

## Example Task: Paper Talk from URL

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal at the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.


In [10]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
Who should read this paper: https://arxiv.org/abs/2308.08155
""",
)

user_proxy (to assistant):


Who should read this paper: https://arxiv.org/abs/2308.08155


--------------------------------------------------------------------------------
assistant (to user_proxy):

```python
# using beautifulsoup to fetch the authors from the paper's webpage on arXiv

from bs4 import BeautifulSoup
import requests

url = 'https://arxiv.org/abs/2308.08155'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

authors = []

# extracting the authors
for author in soup.find_all('meta', attrs={'name': 'citation_author'}):
    authors.append(author['content'])

print(authors)
```


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
['Wu, Qingyun', 'Bansal, Gagan', 'Zhang, Jieyu', 'Wu, Yiran', 'Li, Beibin', 'Zhu, Erkang', 'Jiang, Li', 'Zhang, Xiaoyun', '

ChatResult(chat_id=None, chat_history=[{'content': '\nWho should read this paper: https://arxiv.org/abs/2308.08155\n', 'role': 'assistant'}, {'content': "```python\n# using beautifulsoup to fetch the authors from the paper's webpage on arXiv\n\nfrom bs4 import BeautifulSoup\nimport requests\n\nurl = 'https://arxiv.org/abs/2308.08155'\npage = requests.get(url)\nsoup = BeautifulSoup(page.content, 'html.parser')\n\nauthors = []\n\n# extracting the authors\nfor author in soup.find_all('meta', attrs={'name': 'citation_author'}):\n    authors.append(author['content'])\n\nprint(authors)\n```\n", 'role': 'user'}, {'content': "exitcode: 0 (execution succeeded)\nCode output: \n['Wu, Qingyun', 'Bansal, Gagan', 'Zhang, Jieyu', 'Wu, Yiran', 'Li, Beibin', 'Zhu, Erkang', 'Jiang, Li', 'Zhang, Xiaoyun', 'Zhang, Shaokun', 'Liu, Jiale', 'Awadallah, Ahmed Hassan', 'White, Ryen W', 'Burger, Doug', 'Wang, Chi']\n", 'role': 'assistant'}, {'content': 'The authors of the paper "https://arxiv.org/abs/2308.08155

## Example Task: Chat about Stock Market

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="""Show me the YTD gain of 10 largest technology companies as of today.""",
)